In [1]:
import warnings
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearnex import patch_sklearn, config_context
patch_sklearn()
warnings.filterwarnings('ignore')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit

# custom functions
import config
from utils import DataConnector
from metric import NMAE
from data_processor import WindTransformer, UVTransformer, FeatureTransformer

# model import
import xgboost as xgb
from xgboost import XGBRegressor

# logging
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from mlflow.models import infer_signature

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# remove later

In [12]:
# def get_data():
#     power = pd.read_parquet(config.input_path + "dynamic_report_ewp02_2020_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)
#     train_y = pd.read_parquet(config.input_path + "train_y.parquet").rename({'end_datetime': 'dt'}, axis=1)
#     ldaps = pd.read_parquet(config.input_path + "train_ldaps_gyeongju.parquet")

#     # print("Power: ", power.shape)
#     # print("train_y: ", train_y.shape)
#     # print("LDAPS: ", ldaps.shape)

#     # data slicing
#     power = power[:-3]

#     datas = [power, train_y, ldaps]
#     #datas = [power, ldaps]
#     for d in datas:
#         try:
#             d['dt'] = (pd.to_datetime(d['dt'])
#                         .dt
#                         .tz_convert("Asia/Seoul"))
#         except TypeError:
#             d['dt'] = (pd.to_datetime(d['dt'])
#                         .dt
#                         .tz_localize("Asia/Seoul"))

#     train_y = (train_y.loc[(train_y['plant_name'] == "경주풍력")
#                           & (train_y['dt']).between('2020-01-01', '2022-01-01', inclusive='left')])

#     ldaps = ldaps.loc[ldaps['dt'].between('2020-01-01', '2022-01-01', inclusive='left')]

#     print("Power: ", power.shape, power['dt'].min(), power['dt'].max())
#     print("train_y: ", train_y.shape, train_y['dt'].min(), train_y['dt'].max())
#     print("LDAPS: ", ldaps.shape, ldaps['dt'].min(), ldaps['dt'].max())

#     return power, train_y, ldaps

# 1. Simple Split

In [14]:
if config.mlflow:
    # # create experiment
    # exp = mlflow.set_experiment("windpower_experiment")

    # # start MLflow run
    # with mlflow.start_run(experiment_id=exp.experiment_id):

    #     # Train model
    #     model = xgb.XGBRegressor()

    #     # predict
    #     y_pred = 0
        
    #     # Log model hyperparameters
    #     mlflow.log_params(**config.xgb_params)

    #     # Log performance metrics
    #     mlflow.log_metrics({
    #         "R2 score": r2_score(y_test, y_pred),
    #         "NMAE": NMAE(y_test, y_pred)
    #     })

    #     # Log model
    #     mlflow.sklearn.log_model(model, "xgb")
    pass
else:
    # get data
    scada, train_y, ldaps = get_data()

    # build data pipeline
    print('-' * 50)
    print('Feature Engineering')
    DataPipeline = Pipeline([
        ('uv_transform', UVTransformer('wind_u_10m', 'wind_v_10m')),
        ('wind_transform', WindTransformer('wind_speed', 10, 100, ldaps['surf_rough'].mean())),
        ('feature engineering', FeatureTransformer())
    ])

    # data transform
    ldaps = DataPipeline.fit_transform(ldaps)

    # tmp = pd.merge(ldaps[ldaps['turbine_id'] == 'WTG01'], scada[['dt', 'EnergyProductionActiveEnergyProduction[KWh]']],
    #            on = ['dt'])
    # tmp.columns = tmp.columns.str.replace("[", "_").str.replace("]", "")

    tmp = pd.merge(ldaps[ldaps['turbine_id'] == 'WTG01'], train_y[['dt', 'energy_kwh']])


    # Split train valid
    print('-' * 50)
    print("Train Test Split")
    tmp = tmp.drop(['turbine_id'], axis=1)
    # tmp = tmp.drop(['turbine_id', 'wind_speed'], axis=1)
    target = config.target
    x_train = tmp.loc[tmp['dt'].between('2020-01-01', '2021-01-01', inclusive='left')].drop(['dt', target], axis=1)
    x_test = tmp.loc[tmp['dt'].between('2021-01-01', '2022-01-01', inclusive='left')].drop(['dt', target], axis=1)

    y_train = tmp.loc[tmp['dt'].between('2020-01-01', '2021-01-01', inclusive='left'), target]# / 9
    y_test = tmp.loc[tmp['dt'].between('2021-01-01', '2022-01-01', inclusive='left'), target]

    print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

    # Train model
    print('-' * 50)
    print("Train Model")
    xgb = XGBRegressor(**config.xgb_params)
    xgb.fit(x_train, y_train)

    # predict
    y_pred = xgb.predict(x_test)

    # Scoring
    print("NMAE: ", NMAE(y_test, y_pred))

* dask transform -> 23.5s  
* train_test split (각 1년씩)
* 13.18975

# 2. Target 24h Shift + Simple Split

In [3]:
# SET MLFLOW CONFIG
target = config.target
data_name = "v1" # data name
artifact_path = 'baseline' # model name
registered_model_name= 'tracking-test'

In [4]:
if config.mlflow:
    # get data
    train_y, ldaps = DataConnector().get_data(start_date='2020-01-01',
                                            end_date='2023-01-01',
                                            plant_name='경주풍력')

    # build data pipeline
    print('-' * 50)
    print('Feature Engineering')

    DataPipeline = Pipeline([
        ('uv_transform', UVTransformer('wind_u_10m', 'wind_v_10m')),
        ('wind_transform', WindTransformer('wind_speed', 10, 100, ldaps['surf_rough'].mean())),
        ('feature engineering', FeatureTransformer())
    ])

    # data transform
    ldaps = DataPipeline.fit_transform(ldaps)

    tmp = pd.merge(ldaps[ldaps['turbine_id'] == 'WTG01'], train_y[['dt', 'energy_kwh']])

    # Split train valid
    print('-' * 50)
    print("Train Test Split")
    tmp = tmp.drop(['turbine_id'], axis=1)

    # 24h shift
    tmp['energy_kwh'] = tmp['energy_kwh'].shift(periods=-24)
    tmp = tmp.dropna(axis=0)

    target = config.target

    train = tmp.loc[tmp['dt'].between('2020-01-01', '2021-01-01', inclusive='left')].drop(['dt'], axis=1)
    test = tmp.loc[tmp['dt'].between('2021-01-01', '2022-01-01', inclusive='left')].drop(['dt'], axis=1)
    
    x_train, x_test = train.drop(target, axis=1), test.drop(target, axis=1)
    y_train, y_test = train[target], test[target]

    print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

    # MLFLOW
    print('-' * 50)
    print("model runs & mlflow Logging...")

    dataset = mlflow.data.from_pandas(
        train, name=data_name, targets=config.target
    )

    signature = infer_signature(x_train, y_train)

    # create experiment
    exp = mlflow.set_experiment("windpower_experiment")

    # start MLflow run
    with mlflow.start_run(experiment_id=exp.experiment_id):
        # log input
        mlflow.log_input(dataset, context="training")

        # Train model
        model = xgb.XGBRegressor(random_state=1,
                                 n_jobs=-1,
                                 device='cuda',
                                 objective='reg:absoluteerror',
                                 eval_metric='mae',
                                 **config.xgb_params)

        model.fit(x_train, y_train)

        # predict
        y_pred = model.predict(x_test)
        
        # Log model hyperparameters
        mlflow.log_params(config.xgb_params)

        # Log performance metrics
        mlflow.log_metrics({
            "eval_r2": r2_score(y_test, y_pred),
            "eval_nmae": NMAE(y_test, y_pred)
        })

        # Log model
        mlflow.xgboost.log_model(artifact_path=artifact_path,
                                 xgb_model=model,
                                 signature=signature)
        
        
        print("loss: ", NMAE(y_test, y_pred))

else:
    # get data
    train_y, ldaps = DataConnector.get_data(start_date='2020-01-01',
                                            end_date='2023-01-01',
                                            plant_name='경주풍력')

    # build data pipeline
    print('-' * 50)
    print('Feature Engineering')
    DataPipeline = Pipeline([
        ('uv_transform', UVTransformer('wind_u_10m', 'wind_v_10m')),
        ('wind_transform', WindTransformer('wind_speed', 10, 100, ldaps['surf_rough'].mean())),
        ('feature engineering', FeatureTransformer())
    ])

    # data transform
    ldaps = DataPipeline.fit_transform(ldaps)

    # tmp = pd.merge(ldaps[ldaps['turbine_id'] == 'WTG01'], scada[['dt', 'EnergyProductionActiveEnergyProduction[KWh]']],
    #            on = ['dt'])
    # tmp.columns = tmp.columns.str.replace("[", "_").str.replace("]", "")

    tmp = pd.merge(ldaps[ldaps['turbine_id'] == 'WTG01'], train_y[['dt', 'energy_kwh']])


    # Split train valid
    print('-' * 50)
    print("Train Test Split")
    tmp = tmp.drop(['turbine_id'], axis=1)

    # 24h shift
    tmp['energy_kwh'] = tmp['energy_kwh'].shift(periods=-24)
    tmp = tmp.dropna(axis=0)

    target = config.target
    x_train = tmp.loc[tmp['dt'].between('2020-01-01', '2021-01-01', inclusive='left')].drop(['dt', target], axis=1)
    x_test = tmp.loc[tmp['dt'].between('2021-01-01', '2022-01-01', inclusive='left')].drop(['dt', target], axis=1)

    y_train = tmp.loc[tmp['dt'].between('2020-01-01', '2021-01-01', inclusive='left'), target]
    y_test = tmp.loc[tmp['dt'].between('2021-01-01', '2022-01-01', inclusive='left'), target]

    print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

    # Train model
    print('-' * 50)
    print("Train Model")
    xgb = XGBRegressor(config.xgb_params)
    xgb.fit(x_train, y_train)

    # predict
    y_pred = xgb.predict(x_test)

    # Scoring
    print("NMAE: ", NMAE(y_test, y_pred))

--------------------------------------------------
Feature Engineering
--------------------------------------------------
Train Test Split
(8760, 21) (8759, 21) (8760,) (8759,)
--------------------------------------------------
model runs & mlflow Logging...


2024/10/05 01:43:16 INFO mlflow.tracking.fluent: Experiment with name 'windpower_experiment' does not exist. Creating a new experiment.


loss:  22.45039329501389


# 3. Time Series Split

In [14]:
if config.mlflow:
    # # create experiment
    # exp = mlflow.set_experiment("windpower_experiment")

    # # start MLflow run
    # with mlflow.start_run(experiment_id=exp.experiment_id):

    #     # Train model
    #     model = xgb.XGBRegressor()

    #     # predict
    #     y_pred = 0
        
    #     # Log model hyperparameters
    #     mlflow.log_params(**config.xgb_params)

    #     # Log performance metrics
    #     mlflow.log_metrics({
    #         "R2 score": r2_score(y_test, y_pred),
    #         "NMAE": NMAE(y_test, y_pred)
    #     })

    #     # Log model
    #     mlflow.sklearn.log_model(model, "xgb")
    pass
else:
    # get data
    scada, train_y, ldaps = get_data()

    # build data pipeline
    print('-' * 50)
    print('Feature Engineering')
    DataPipeline = Pipeline([
        ('uv_transform', UVTransformer('wind_u_10m', 'wind_v_10m')),
        ('wind_transform', WindTransformer('wind_speed', 10, 100, ldaps['surf_rough'].mean())),
        ('feature engineering', FeatureTransformer())
    ])

    # data transform
    ldaps = DataPipeline.fit_transform(ldaps)

    # tmp = pd.merge(ldaps[ldaps['turbine_id'] == 'WTG01'], scada[['dt', 'EnergyProductionActiveEnergyProduction[KWh]']],
    #            on = ['dt'])
    # tmp.columns = tmp.columns.str.replace("[", "_").str.replace("]", "")

    tmp = pd.merge(ldaps[ldaps['turbine_id'] == 'WTG01'], train_y[['dt', 'energy_kwh']])


    # Split train valid
    print('-' * 50)
    print("Train Test Split")
    tmp = tmp.drop(['turbine_id', 'dt'], axis=1)

    # # 24h shift
    # tmp['energy_kwh'] = tmp['energy_kwh'].shift(periods=24)
    # tmp = tmp.dropna(axis=0)

    target = config.target
    n_splits = 5
    tscv = TimeSeriesSplit(n_splits=n_splits,  
                           test_size=24*30, # test size는 하루? 아니면 그냥 길게?
                           gap=24*30) # 한달의 gap


    nmae_lists = []
    idx = 0
    for train_index, test_index in tscv.split(tmp):
        train = tmp.iloc[train_index]
        validate = tmp.iloc[test_index]

        print('-' * 50)
        print("Fold: ", idx)
        x_train, y_train = train.drop(target, axis=1), train[target]
        x_test, y_test = validate.drop(target, axis=1), validate[target]

        print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

        # Train model
        print('-' * 50)
        print("Train Model")
        xgb = XGBRegressor(**config.xgb_params)
        xgb.fit(x_train, y_train)

        # predict
        y_pred = xgb.predict(x_test)

        # Scoring
        print("NMAE: ", NMAE(y_test, y_pred))
        nmae_lists.append(NMAE(y_test, y_pred))
        idx += 1

    print("Mean NMAE: ", np.mean(nmae_lists))
    

Power:  (52592, 29)
train_y:  (52608, 4)
LDAPS:  (235818, 15)
Power:  (52589, 29) 2020-01-01 00:00:00+09:00 2020-12-31 23:50:00+09:00
train_y:  (17543, 4) 2020-01-01 01:00:00+09:00 2021-12-31 23:00:00+09:00
LDAPS:  (157671, 15) 2020-01-02 00:00:00+09:00 2021-12-31 23:00:00+09:00
--------------------------------------------------
Feature Engineering
--------------------------------------------------
Train Test Split
--------------------------------------------------
Fold:  0
(13199, 21) (720, 21) (13199,) (720,)
--------------------------------------------------
Train Model
NMAE:  12.68159195131598
--------------------------------------------------
Fold:  1
(13919, 21) (720, 21) (13919,) (720,)
--------------------------------------------------
Train Model
NMAE:  13.22209786677866
--------------------------------------------------
Fold:  2
(14639, 21) (720, 21) (14639,) (720,)
--------------------------------------------------
Train Model
NMAE:  11.936203140497078
----------------------

# 4. Target 24h Shift + Time Series Split

In [6]:
if config.mlflow:
    # # create experiment
    # exp = mlflow.set_experiment("windpower_experiment")

    # # start MLflow run
    # with mlflow.start_run(experiment_id=exp.experiment_id):

    #     # Train model
    #     model = xgb.XGBRegressor()

    #     # predict
    #     y_pred = 0
        
    #     # Log model hyperparameters
    #     mlflow.log_params(**config.xgb_params)

    #     # Log performance metrics
    #     mlflow.log_metrics({
    #         "R2 score": r2_score(y_test, y_pred),
    #         "NMAE": NMAE(y_test, y_pred)
    #     })

    #     # Log model
    #     mlflow.sklearn.log_model(model, "xgb")
    pass
else:
    # get data
    scada, train_y, ldaps = get_data()

    # build data pipeline
    print('-' * 50)
    print('Feature Engineering')
    DataPipeline = Pipeline([
        ('uv_transform', UVTransformer('wind_u_10m', 'wind_v_10m')),
        ('wind_transform', WindTransformer('wind_speed', 10, 100, ldaps['surf_rough'].mean())),
        ('feature engineering', FeatureTransformer())
    ])

    # data transform
    ldaps = DataPipeline.fit_transform(ldaps)

    # tmp = pd.merge(ldaps[ldaps['turbine_id'] == 'WTG01'], scada[['dt', 'EnergyProductionActiveEnergyProduction[KWh]']],
    #            on = ['dt'])
    # tmp.columns = tmp.columns.str.replace("[", "_").str.replace("]", "")

    tmp = pd.merge(ldaps[ldaps['turbine_id'] == 'WTG01'], train_y[['dt', 'energy_kwh']])


    # Split train valid
    print('-' * 50)
    print("Train Test Split")
    tmp = tmp.drop(['turbine_id', 'dt'], axis=1)

    # 24h shift
    tmp['energy_kwh'] = tmp['energy_kwh'].shift(periods=-24)
    tmp = tmp.dropna(axis=0)

    target = config.target
    n_splits = 5
    tscv = TimeSeriesSplit(n_splits=n_splits,  
                           test_size=24*30, # test size는 하루? 아니면 그냥 길게?
                           gap=24*30) # 한달의 gap

    nmae_lists = []
    idx = 0
    for train_index, test_index in tscv.split(tmp):
        train = tmp.iloc[train_index]
        validate = tmp.iloc[test_index]

        print('-' * 50)
        print("Fold: ", idx)
        x_train, y_train = train.drop(target, axis=1), train[target]
        x_test, y_test = validate.drop(target, axis=1), validate[target]

        print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

        # Train model
        print('-' * 50)
        print("Train Model")
        xgb = XGBRegressor(**config.xgb_params)
        xgb.fit(x_train, y_train)

        # predict
        y_pred = xgb.predict(x_test)

        # Scoring
        print("NMAE: ", NMAE(y_test, y_pred))
        nmae_lists.append(NMAE(y_test, y_pred))
        idx += 1

    print("Mean NMAE: ", np.mean(nmae_lists))

Power:  (52592, 29)
train_y:  (52608, 4)
LDAPS:  (235818, 15)
Power:  (52589, 29) 2020-01-01 00:00:00+09:00 2020-12-31 23:50:00+09:00
train_y:  (17543, 4) 2020-01-01 01:00:00+09:00 2021-12-31 23:00:00+09:00
LDAPS:  (157671, 15) 2020-01-02 00:00:00+09:00 2021-12-31 23:00:00+09:00
--------------------------------------------------
Feature Engineering
--------------------------------------------------
Train Test Split
--------------------------------------------------
Fold:  0
(13175, 21) (720, 21) (13175,) (720,)
--------------------------------------------------
Train Model
NMAE:  18.619867066817456
--------------------------------------------------
Fold:  1
(13895, 21) (720, 21) (13895,) (720,)
--------------------------------------------------
Train Model
NMAE:  19.47732427927735
--------------------------------------------------
Fold:  2
(14615, 21) (720, 21) (14615,) (720,)
--------------------------------------------------
Train Model
NMAE:  21.969813565905863
---------------------

# 정리
(WTG01)
* Simple Split: 13.189
* Target 24h Shift + Simple Split: 21.711 
* Time Series Split(5 Fold): 13.268 (average)
* Target 24h Shift + Simple Split(5 Fold): 21.209 (average)

(WTG02)  
* Target 24h Shift + Simple Split: 21.5614

([WTG01 + WTG02]/2)
* Target 24h Shift + Simple Split: 21.2430